In [1]:
import pyodbc
import pandas as pd
import numpy as np 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
## 라이브러리 불러오기
'''메인 라이브러리'''
import numpy as np
import pandas as pd
import os, time, re
import pickle, gzip

'''시각화 관련 라이브러리'''
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
import matplotlib as mpl

%matplotlib inline

'''데이터 준비 및 모델 평가 관련 라이브러리'''
from sklearn import preprocessing as pp
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc, roc_auc_score

'''알고리즘 관련 라이브러리'''
import lightgbm as lgb

'''텐서플로 및 케라스 관련 라이브러리'''
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Dropout
from keras.layers import BatchNormalization, Input, Lambda
from keras import regularizers
from keras.losses import mse, binary_crossentropy

from collections import Counter
import numpy as np
import seaborn as sns
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from collections import Counter

import tensorflow as tf
import tensorflow.keras.layers as layers
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def encode_features(dataDF):
    features = ['userip','phone','identikey','CPID','dnGrade','age','cs','isNew','amtlimit']
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(dataDF[feature])
        dataDF[feature] = le.transform(dataDF[feature])
        # decode
        # le.inverse_transform(dataDF[feature])
        
    return dataDF

df = pd.read_csv('C://Users/USER/20210906/credit_card/creditcard.csv')

#df = df.drop(["miAMT2th","miAMT3th","isCTR","dnpoint","lastUsedCP"],axis=1)

#df = encode_features(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [3]:
df.head(10)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [4]:
# X_train, X_test, y_train, y_test = train_test_split(df.drop('isSuspect', axis=1), df['isSuspect'], 
#                                                     test_size=0.5, random_state=0)

In [5]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

X_train, X_test,y_train, y_test= train_test_split(X, y,train_size=0.7, 
                                                    test_size=0.3, random_state=121)

# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=0)
# X_train, y_train = smote.fit_resample(X_train, y_train)
# start_time = time.time()

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve

# 수정된 get_clf_eval() 함수 
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    #print('오차 행렬')
    #print(confusion)
    
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))


In [7]:
from sklearn.ensemble import RandomForestClassifier

start_time = time.time()

# 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
rf_clf = RandomForestClassifier(n_estimators=70, max_depth=18, min_samples_leaf=3, \
                                 min_samples_split=2, criterion='entropy', random_state=0, verbose=1,  n_jobs=-1, warm_start=True)
rf_clf.fit(X_train , y_train)
#%time print(rf_clf.fit(X_train , y_train))
print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))
pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test , pred)
pred_proba = rf_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test , pred, pred_proba)

cm = confusion_matrix(y_test, pred)

print(cm)

# true/false positives/negatives
(tp, fp, 
 fn, tn) = cm.flatten()

FPR = fp / (tp+tn) 
FNR = fn / (tp+fn) 

print(FPR, FNR)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   14.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  70 out of  70 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.


수행 시간: 14.3 초 


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done  70 out of  70 | elapsed:    0.0s finished


정확도: 0.9995, 정밀도: 0.9453, 재현율: 0.7806,    F1: 0.8551, AUC:0.9450
[[85281     7]
 [   34   121]]
8.19652935528442e-05 0.0003985231202016058


In [8]:
# 사이킷런 래퍼 XGBoost 클래스인 XGBClassifier 임포트
from xgboost import XGBClassifier

evals = [(X_test, y_test)]

start_time = time.time()

xgb_wrapper = XGBClassifier(n_estimators=400, learning_rate=0.1, max_depth=3)

xgb_wrapper.fit(X_train , y_train,  early_stopping_rounds=100,eval_set=evals, eval_metric="logloss",  verbose=True)
print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))
w_preds = xgb_wrapper.predict(X_test)
w_pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test ,w_preds, w_pred_proba)

cm = confusion_matrix(y_test, w_preds)
print(cm)
# true/false positives/negatives
(tp, fp, 
 fn, tn) = cm.flatten()

오탐률 = fp / (fp+tn)
미탐률 = fn / (tp+fn) 

print(오탐률, 미탐률)

[0]	validation_0-logloss:0.59835
[1]	validation_0-logloss:0.52081
[2]	validation_0-logloss:0.45606
[3]	validation_0-logloss:0.40134
[4]	validation_0-logloss:0.35464
[5]	validation_0-logloss:0.31441
[6]	validation_0-logloss:0.27949
[7]	validation_0-logloss:0.24906
[8]	validation_0-logloss:0.22240
[9]	validation_0-logloss:0.19889
[10]	validation_0-logloss:0.17818
[11]	validation_0-logloss:0.15982
[12]	validation_0-logloss:0.14353
[13]	validation_0-logloss:0.12904
[14]	validation_0-logloss:0.11614
[15]	validation_0-logloss:0.10463
[16]	validation_0-logloss:0.09433
[17]	validation_0-logloss:0.08512
[18]	validation_0-logloss:0.07687
[19]	validation_0-logloss:0.06946
[20]	validation_0-logloss:0.06282
[21]	validation_0-logloss:0.05685
[22]	validation_0-logloss:0.05150
[23]	validation_0-logloss:0.04668
[24]	validation_0-logloss:0.04234
[25]	validation_0-logloss:0.03844
[26]	validation_0-logloss:0.03492
[27]	validation_0-logloss:0.03175
[28]	validation_0-logloss:0.02890
[29]	validation_0-loglos

[238]	validation_0-logloss:0.00255
[239]	validation_0-logloss:0.00255
[240]	validation_0-logloss:0.00255
[241]	validation_0-logloss:0.00254
[242]	validation_0-logloss:0.00255
[243]	validation_0-logloss:0.00255
[244]	validation_0-logloss:0.00255
[245]	validation_0-logloss:0.00255
[246]	validation_0-logloss:0.00255
[247]	validation_0-logloss:0.00255
[248]	validation_0-logloss:0.00254
[249]	validation_0-logloss:0.00254
[250]	validation_0-logloss:0.00254
[251]	validation_0-logloss:0.00254
[252]	validation_0-logloss:0.00254
[253]	validation_0-logloss:0.00254
[254]	validation_0-logloss:0.00254
[255]	validation_0-logloss:0.00254
[256]	validation_0-logloss:0.00254
[257]	validation_0-logloss:0.00254
[258]	validation_0-logloss:0.00254
[259]	validation_0-logloss:0.00254
[260]	validation_0-logloss:0.00254
[261]	validation_0-logloss:0.00254
[262]	validation_0-logloss:0.00254
[263]	validation_0-logloss:0.00254
[264]	validation_0-logloss:0.00254
[265]	validation_0-logloss:0.00254
[266]	validation_0-l

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')
# GBM 수행 시간 측정을 위함. 시작 시간 설정.
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train , y_train)
print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))
gb_pred = gb_clf.predict(X_test)
pred_proba = gb_clf.predict_proba(X_test)[:, 1]
gb_accuracy = accuracy_score(y_test, gb_pred)

get_clf_eval(y_test ,gb_pred, pred_proba)

cm = confusion_matrix(y_test, gb_pred)
print(cm)
# true/false positives/negatives
(tp, fp, 
 fn, tn) = cm.flatten()

오탐률 = fp / (fp+tn)
미탐률 = fn / (tp+fn) 

print(오탐률, 미탐률)

수행 시간: 267.9 초 
정확도: 0.9990, 정밀도: 0.8586, 재현율: 0.5484,    F1: 0.6693, AUC:0.7224
[[85274    14]
 [   70    85]]
0.1414141414141414 0.0008202099737532808


In [10]:
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np

In [11]:
start_time = time.time()

params = {
    'application': 'binary', 
    'boosting': 'gbdt',
    'num_iterations': 8, 
    'device': 'gpu', 
    'max_bin': 250, 
    'metric' : 'binary_logloss',
    'n_estimators': 100, 
    'learning_rate': 0.03, 
    'num_leaves': 960, 
    'max_depth': -1, 
    'min_data_in_leaf': 4900, 
    'lambda_l1': 10, 
    'lambda_l2': 45, 
    'min_gain_to_split': 5.252490881397367, 
    'bagging_fraction': 0.5, 
    'bagging_freq': 1, 
    'feature_fraction': 0.7
}


start_time = time.time()

# 앞서 XGBoost와 동일하게 n_estimators는 400 설정. 
lgbm_wrapper = LGBMClassifier(
    boosting_type= 'gbdt', 
    objective = 'binary', 
    device = 'gpu',
    max_depth = params['max_depth'],
    max_bin = params['max_bin'], 
    metric = params['metric'],
    application = 'binary',
    n_estimators = params['n_estimators'],
   # random_state = 121,
    num_leaves = params['num_leaves'], 
    #lambda_l1 = params['lambda_l1'], 
    #lambda_l2 = params['lambda_l2'], 
    #min_gain_to_split = params['min_gain_to_split'], 
    #bagging_fraction = params['bagging_fraction'], 
    #bagging_freq = params['bagging_freq'], 
    #feature_fraction = params['feature_fraction'],
    learning_rate = params['learning_rate']
    #num_iterations = params['num_iterations']
    )

# LightGBM도 XGBoost와 동일하게 조기 중단 수행 가능. 
evals = [(X_test, y_test)]
lgbm_wrapper.fit(X_train , y_train, early_stopping_rounds=100, eval_metric="logloss", \
                 eval_set=evals, verbose=True)
print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))
preds = lgbm_wrapper.predict(X_test)
pred_proba = lgbm_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)

cm = confusion_matrix(y_test, preds)
print(cm)
# true/false positives/negatives
(tp, fp, 
 fn, tn) = cm.flatten()

오탐률 = fp / (fp+tn)
미탐률 = fn / (tp+fn) 

print(오탐률, 미탐률)

[1]	valid_0's binary_logloss: 0.00467122
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.00467947
[3]	valid_0's binary_logloss: 0.0046474
[4]	valid_0's binary_logloss: 0.00459939
[5]	valid_0's binary_logloss: 0.00455491
[6]	valid_0's binary_logloss: 0.00451539
[7]	valid_0's binary_logloss: 0.00447591
[8]	valid_0's binary_logloss: 0.00443311
[9]	valid_0's binary_logloss: 0.00439069
[10]	valid_0's binary_logloss: 0.00434473
[11]	valid_0's binary_logloss: 0.00430785
[12]	valid_0's binary_logloss: 0.00426609
[13]	valid_0's binary_logloss: 0.00422657
[14]	valid_0's binary_logloss: 0.00418834
[15]	valid_0's binary_logloss: 0.00415286
[16]	valid_0's binary_logloss: 0.00411821
[17]	valid_0's binary_logloss: 0.00408479
[18]	valid_0's binary_logloss: 0.00405359
[19]	valid_0's binary_logloss: 0.00402302
[20]	valid_0's binary_logloss: 0.00399187
[21]	valid_0's binary_logloss: 0.00396229
[22]	valid_0's binary_logloss: 0.00393529
[23]	valid_0's binary_lo

In [12]:
# 필요 라이브러리 import
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

In [13]:
import pandas as pd

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# 개별 모델은 로지스틱 회귀와 KNN 임. 
lr_clf = LogisticRegression()
#knn_clf = KNeighborsClassifier(n_neighbors=8)
dt_clf = DecisionTreeClassifier(random_state=0)
C = 1                         # SVM의 regularization parameter
svm_clf = svm.SVC(C=C, max_iter = 100, probability=True)
nb_clf = GaussianNB()

# 개별 모델을 소프트 보팅 기반의 앙상블 모델로 구현한 분류기 
vo_clf = VotingClassifier( estimators=[('dt_clf',dt_clf),('svm_clf',svm_clf),('nb_clf',nb_clf)] , voting='soft' )

# VotingClassifier 학습/예측/평가. 
vo_clf.fit(X_train , y_train)
pred = vo_clf.predict(X_test)
pred_proba = vo_clf.predict_proba(X_test)[:, 1]
confusion = confusion_matrix( y_test, pred)
accuracy = accuracy_score(y_test , pred)
precision = precision_score(y_test , pred)
recall = recall_score(y_test , pred)
f1 = f1_score(y_test,pred)
roc_auc = roc_auc_score(y_test, pred_proba)

print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

start_time = time.time()

# 개별 모델의 학습/예측/평가.
classifiers = [dt_clf, svm_clf, nb_clf]
for classifier in classifiers:
    classifier.fit(X_train , y_train)
    pred = classifier.predict(X_test)
    pred_proba = classifier.predict_proba(X_test)[:, 1]
    
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    cm = confusion_matrix(y_test, pred)
    print(cm)
    # true/false positives/negatives
    (tp, fp, 
     fn, tn) = cm.flatten()

    오탐률 = fp / (fp+tn)
    미탐률 = fn / (tp+fn) 

    print(오탐률, 미탐률)
    
    class_name= classifier.__class__.__name__
    print('{0} 정확도: {1:.4f}, 정밀도: {2:.4f}, 재현율: {3:.4f},\
    F1: {4:.4f}, AUC:{5:.4f}'.format(class_name, accuracy, precision, recall, f1, roc_auc))
    
print("수행 시간: {0:.1f} 초 ".format(time.time() - start_time))    



정확도: 0.9993, 정밀도: 0.9505, 재현율: 0.6194,    F1: 0.7500, AUC:0.9217
[[85246    42]
 [   35   120]]
0.25925925925925924 0.00041040794549782484
DecisionTreeClassifier 정확도: 0.9991, 정밀도: 0.7407, 재현율: 0.7742,    F1: 0.7571, AUC:0.8869
[[25249 60039]
 [   75    80]]
0.9986693058766779 0.002961617438003475
SVC 정확도: 0.2964, 정밀도: 0.0013, 재현율: 0.5161,    F1: 0.0027, AUC:0.3731
[[84734   554]
 [   54   101]]
0.8458015267175573 0.0006368825777232627
GaussianNB 정확도: 0.9929, 정밀도: 0.1542, 재현율: 0.6516,    F1: 0.2494, AUC:0.9611
수행 시간: 53.4 초 
